# Notebook E-tivity 1 CE4021

Student name: Mitchell de Bruyn

Student ID: 23296828

<hr style="border:2px solid gray"> </hr>

## Imports

In [1]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style="border:2px solid gray"> </hr>

## Task

You may add as many cells as you require to complete this task. Refer to the Rubric for E-tivity 1 to see how you will be assessed.

# Symbolic calculation of the derivative

In order to perform the symbolic differentiation of a single varaible, we will use the basic rule:

$$
\frac{d}{dx} ax^n = n\times ax^{n-1}
$$

Representing a polynomial as a list in reverse exponential order such that the list values are the coefficients and the index positions are the powers.
In this way the following polynomial can be written as:

$$
3x^2+1x^1+4 \to [4,1,3]
$$

These solutions makes a few important assumuptions:
1. All equations are fully expanded and simplified.
2. All exponents are positive integers.

## Pretty poly 

This is only a helper function to present answers in a nice sting format

In [1]:
def pretty_poly(f: list) -> str:
    """
    Helper function that will take a list representing a polynomial and return a more human readable string
    """
    ans = ""
    for exponent, value in enumerate(f):
        # skip exponents where the coefficient is 0
        if value == 0:
            continue

        # Check if we already have a partial answer and
        # check if the previously added term did not have a - sign
        if ans and ans[0] != '-':
            ans = "+" + ans

        # x^0 == 1, so don't bother writing the x for 0
        # only write the coefficient
        if exponent == 0:
            ans = f"{value}" + ans
            continue

        # x^1 can be simplified to just x
        if exponent == 1:
            if value == 1:
                ans = f"x" + ans
            else:
                ans = f"{value}x" + ans

        else: # this is the general case so adding ax^n to the front of the string
            if value == 1:
                ans = f"x^{exponent}" + ans
            else:
                ans = f"{value}x^{exponent}" + ans

    if not ans:
        ans = "0"
    
    return ans

## Validate helper function

Just a quick test to see we get reasonable displays.

1. $ 5x^3 $
2. $ x^2 $
3. $ 3x + 5 $
4. $ 3x^2 + 5 $

In [2]:
print(f"1. {pretty_poly([0, 0, 0, 5])}")
print(f"2. {pretty_poly([0, 0, 1])}")
print(f"3. {pretty_poly([5, 3])}")
print(f"4. {pretty_poly([5, 0, 3])}")

1. 5x^3
2. x^2
3. 3x+5
4. 3x^2+5


## differentiate

This function uses enumerate to walk through the array and apply the basic rule:
$$
\frac{d}{dx} ax^n = n\times ax^{n-1}
$$


In [3]:
def differentiate(f: list) -> list:
    """
    This Function will do symbolic differentiation of a polynomial represented by list.

    Input:
    f - list of integers representing a polynomial

    Output:
    list of integers representing the differentiated polynomial
    """
    ans = [0]*len(f)
    for exponent, value in enumerate(f):   
        # x^0 == 1 in this case the coefficient is a constant in this function with no x dependance
        # so the constant is dropped during differentiation.
        if exponent != 0: 
            ans[exponent-1] = value*exponent
            
    return ans

In [4]:
def nice_test_differentiate(f: list):
    """
    Nice Test is just a nice wrapper to print in a pretty presentation
    """
    print(f"d/dx[{pretty_poly(f)}] = {pretty_poly(differentiate(f))}")

# Testing

Here are the basic test cases:
1. $ \frac{d}{dx}5x^3 = 15x^2$
2. $ \frac{d}{dx}3x + 5 = 3$
3. $ \frac{d}{dx}3x^2 + 5 = 6x$
4. $ \frac{d}{dx}4x^2-1x+3 = 8x-1$
5. $ \frac{d}{dx}8x^3+x^2-7x+2 = 24x^2+2x-7$
6. $ \frac{d}{dx}3x^4-4x^3+7x^2+6x+6 = 12x^3-12x^2+14x+6$
7. $ \frac{d}{dx}-8x^3-1x^2-6x+1 = -24x^2-2x-6$

In [10]:
nice_test_differentiate([0, 0, 0, 5])
nice_test_differentiate([5, 3])
nice_test_differentiate([5, 0, 3])
nice_test_differentiate([3, -1, 4])
nice_test_differentiate([2, -7, 1, 8])
nice_test_differentiate([6, 6, 7, -4, 3, 0])
nice_test_differentiate([1, -6, -1, -8])

d/dx[5x^3] = 15x^2
d/dx[3x+5] = 3
d/dx[3x^2+5] = 6x
d/dx[4x^2-1x+3] = 8x-1
d/dx[8x^3+x^2-7x+2] = 24x^2+2x-7
d/dx[3x^4-4x^3+7x^2+6x+6] = 12x^3-12x^2+14x+6
d/dx[-8x^3-1x^2-6x+1] = -24x^2-2x-6


# Evaluate a polynomial for a given value of its variable

For this section I'm going to keep working with my polynomial in the reverse list representation as above:

In [11]:
def evaluate_poly_at(poly: list, x: float) -> float:
    ans: float = 0
    for exponent, value in enumerate(poly):
        ans += value * x**exponent
    return ans

In [12]:
def nice_test_evaluate(poly: list, x: float):
    print("f(x) = %-15s;  f(%s) = %s" % (pretty_poly(poly), x, evaluate_poly_at(poly, x)))

In [13]:
nice_test_evaluate([0], 6)
nice_test_evaluate([1], 6)
nice_test_evaluate([0,1], 7)
nice_test_evaluate([0,0,2], 4)
nice_test_evaluate([3,5,8], 3)

f(x) = 0              ;  f(6) = 0
f(x) = 1              ;  f(6) = 1
f(x) = x              ;  f(7) = 7
f(x) = 2x^2           ;  f(4) = 32
f(x) = 8x^2+5x+3      ;  f(3) = 90


# Numerical integration over a fixed interval

There are some common methods for numerical integration (Midpoint, Trapezoid, Simpson's rule).

I'm just quickly going to use the midpoint rule. Here we use divide the area under the function curve up into a number of rectangals.
Depending on where the rectangle touches the line, the area of the rectangle will be either an over or under estimate of the true area. To improve this estimate we will use a midoint, so the left half of may be a bit over and the right part may be a bit under, but the estimate will hopefully cancel out.

The area of a rectangle is given my the height x width.

In this application I'm going to set the height as the value returned by the function and the width of each rectangle will be the range for the integral devided by the number of steps.

In [20]:
def midpoint_integration(f, a: float, b: float, n: int) -> float:
    dx = float(b - a)/float(n) # I don't trust integer devision, it's better in python 3 but it's broken too many of my solutions!
    
    area: float = 0
    for i in range(n):
        x = a + dx * (i+0.5) # add 0.5 to use the midpoint instead of the 
        h = f(x)
        area += h * dx

    return area

## Test midpoint integration

This basic test is going to consider the integral under the curve for $f(x) = x^2$

$$
\int_a^b{x^2} = \frac{1}{3}x^3 + C
$$

For the range it will consider $0 \to 1$

$$
\int_0^1{x^2} = [\frac{1}{3}[1]^3 + C] - [\frac{1}{3}[0] + C] = \frac{1}{3}
$$


In [22]:
def f(x: float):
    return x**2

print(midpoint_integration(f, 0, 1, 5))
print(midpoint_integration(f, 0, 1, 10))
print(midpoint_integration(f, 0, 1, 20))
print(midpoint_integration(f, 0, 1, 100))
print(midpoint_integration(f, 0, 1, 100000))
print(midpoint_integration(f, 0, 1, 100000000))

0.33000000000000007
0.3325
0.33312500000000006
0.33332500000000004
0.3333333333249995
0.3333333333333373


In [2]:
# import time # nothing to see here ;) 
# def benchmark_integration():
#     for i in range(1, 10):
#         start = time.time()
#         print(f"{10**i} steps = {midpoint_integration(f, 0, 1, 10**i)} in ", end = '')
#         print(f"{time.time() - start} seconds")

# benchmark_integration()
# 10 steps = 0.3325 in 0.0001811981201171875 seconds
# 100 steps = 0.33332500000000004 in 0.00011944770812988281 seconds
# 1000 steps = 0.3333332499999999 in 0.0010640621185302734 seconds
# 10000 steps = 0.33333333250000036 in 0.01449275016784668 seconds
# 100000 steps = 0.3333333333249995 in 0.059927940368652344 seconds
# 1000000 steps = 0.3333333333332426 in 0.21194934844970703 seconds
# 10000000 steps = 0.3333333333333564 in 1.6543059349060059 seconds
# 100000000 steps = 0.3333333333333373 in 16.638136386871338 seconds
# 1000000000 steps = 0.3333333333332523 in 175.94883060455322 seconds

<hr style="border:2px solid gray"> </hr>

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
- Compare your approach to alternative (peer) approaches.
- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.